In [2]:
import pandas as pd
import folium
from haversine import haversine
import numpy as np
import seaborn as sns
import json
import os
from copy import deepcopy 
import geopandas as gpd
from shapely.geometry import Point
sns.set()

In [3]:
# Local
# Font
import matplotlib
from matplotlib import font_manager, rc
import platform
if platform.system()=="Windows":
    font_name=font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

import warnings
warnings.filterwarnings("ignore")

In [4]:
path = "D:\COMPAS\SBJ_2012_001"

__csv files__

In [97]:
주정차단속 = pd.read_csv(os.path.join(path,'1.오산시_주정차단속(2018~2020).csv'),encoding="utf-8")
유동인구 = pd.read_csv(os.path.join(path,'8.오산시_유동인구(2019).csv'),encoding="utf-8")
어린이보호구역 = pd.read_csv(os.path.join(path,'9.오산시_어린이보호구역.csv'),encoding="utf-8")
학교위치정보 = pd.read_csv(os.path.join(path,'10.오산시_학교위치정보.csv'),encoding="utf-8")
어린이집_유치원현황 = pd.read_csv(os.path.join(path,'13.오산시_어린이집_유치원현황.csv'),encoding="utf-8")
기상데이터 = pd.read_csv(os.path.join(path,'14.오산시_기상데이터(2010~2019).csv'),encoding="utf-8")
무인교통단속카메라 = pd.read_csv(os.path.join(path,'15.오산시_무인교통단속카메라.csv'),encoding="utf-8")
도로안전표지 = pd.read_csv(os.path.join(path,'16.오산시_도로안전표지표준데이터.csv'),encoding="utf-8")
과속방지턱 = pd.read_csv(os.path.join(path,'18.오산시_과속방지턱표준데이터.csv'),encoding="utf-8")
CCTV설치현황 = pd.read_csv(os.path.join(path,'20.오산시_CCTV설치현황.csv'),encoding="utf-8")
버스정류장 = pd.read_csv(os.path.join(path,'22.오산시_버스정류장.csv'),encoding="utf-8")
시간대별_추정교통량 = pd.read_csv(os.path.join(path,'24.평일_전일,시간대별_오산시_추정교통량_Level6.csv'),encoding="utf-8")
혼잡빈도강도 = pd.read_csv(os.path.join(path,'25.평일_전일_오산시_혼잡빈도강도_Level6.csv'),encoding="utf-8")
혼잡시간강도 = pd.read_csv(os.path.join(path,'26.평일_전일_오산시_혼잡시간강도_Level6.csv'),encoding="utf-8")
체육시설현황 = pd.read_csv(os.path.join(path,'29.오산시_체육시설현황.csv'),encoding="utf-8")
학원_및_교습소_현황 = pd.read_csv(os.path.join(path,'30.오산시_학원 및 교습소 현황.csv'),encoding="utf-8")

__geojson files__

In [89]:
어린이교통사고_격자 = gpd.read_file(os.path.join(path,'2.오산시_어린이교통사고_격자.geojson'))
차량등록현황_격자 = gpd.read_file(os.path.join(path,'3.오산시_차량등록현황_격자.geojson'))
거주인구격자_총인구 = gpd.read_file(os.path.join(path,'4.오산시_연령별_거주인구격자(총인구).geojson'))
거주인구격자_유소년 = gpd.read_file(os.path.join(path,'5.오산시_연령별_거주인구격자(유소년).geojson'))
거주인구격자_생산가능 = gpd.read_file(os.path.join(path,'6.오산시_연령별_거주인구격자(생산가능인구).geojson'))
거주인구격자_고령 = gpd.read_file(os.path.join(path,'7.오산시_연령별_거주인구격자(고령).geojson'))
초등학교_통학구 = gpd.read_file(os.path.join(path,'11.오산시_초등학교_통학구.geojson'))
중학교_학군 = gpd.read_file(os.path.join(path,'12.오산시_중학교_학군.geojson'))
횡단보도 = gpd.read_file(os.path.join(path,'17.오산시_횡단보도.geojson'))
신호등 = gpd.read_file(os.path.join(path,'19.오산시_신호등.geojson'))
인도 = gpd.read_file(os.path.join(path,'21.오산시_인도.geojson'))
상세도로망 = gpd.read_file(os.path.join(path,'23.오산시_상세도로망_LV6.geojson'))
도로명주소_건물 = gpd.read_file(os.path.join(path,'27.오산시_도로명주소_건물.geojson'))
건물연면적_격자 = gpd.read_file(os.path.join(path,'28.오산시_건물연면적_격자.geojson'))
법정경계_시군구 = gpd.read_file(os.path.join(path,'31.오산시_법정경계(시군구).geojson'))
행정경계_읍면동 = gpd.read_file(os.path.join(path,'32.오산시_행정경계(읍면동).geojson'))
법정경계_읍면동 = gpd.read_file(os.path.join(path,'33.오산시_법정경계(읍면동).geojson'))
지적도 = gpd.read_file(os.path.join(path,'34.오산시_지적도.geojson'))

### 계속 쓸 기본 틀 

In [6]:
data = 어린이교통사고_격자[어린이교통사고_격자.accident_cnt>0].reset_index().copy()
data = data.drop(["index"],axis=1)
data.head(3)

,gid,accident_cnt,geometry
0,다사561083,1,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


### feature : 격자 내부 유동인구 합

In [7]:
유동인구.drop(["STD_YM"],axis=1).groupby(['lon','lat']).mean()
관심시간대 = ["TMST_08","TMST_09","TMST_10","TMST_12","TMST_13","TMST_14","TMST_15","TMST_16","TMST_17","TMST_18"]
전체평균유동인구 = 유동인구.drop(["STD_YM"],axis=1).groupby(['lon','lat']).mean()[관심시간대].mean(axis=1).reset_index()
전체평균유동인구.head(3)

,lon,lat,0
0,126.995142,37.177153,0.546000
1,126.995145,37.176703,0.882000
2,126.995148,37.176252,0.219429


In [8]:
전체평균유동인구.shape

(12402, 3)

In [9]:
전체평균유동인구[전체평균유동인구[0]>1].shape

(6641, 3)

In [19]:
def ppl_inside(geo_obj):
    
    global 전체평균유동인구
    
    ret_cnt = 0
    
    for (lon,lat,cnt) in 전체평균유동인구.iloc:
        current_point = Point(lon,lat)
        if geo_obj.contains(current_point):
            ret_cnt+=cnt
    return ret_cnt

data['격자내 유동인구'] = data['geometry'].apply(ppl_inside)
data.head(3)

,gid,accident_cnt,geometry,격자내 유동인구
0,다사561083,1,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ...",43.435676
1,다사564096,1,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ...",9.633417
2,다사566102,1,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ...",0.215500


### feature : 격자 중심에서 200m 주변 유소년 총인구

In [71]:
data = gpd.read_file(os.path.join(path,'GDF_for_linear_regression.geojson'))
data.head()

,gid,accident_cnt,격자내 유동인구,격자내 유소년 인구,격자내 학원 수,geometry
0,다사561083,1,43.435676,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,9.633417,0.0,0,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,0.215500,0.0,0,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."
3,다사583093,1,0.662000,0.0,0,"MULTIPOLYGON (((127.03022 37.18154, 127.03021 ..."
4,다사584072,1,8.104333,0.0,0,"MULTIPOLYGON (((127.03146 37.16262, 127.03145 ..."


In [7]:
거주인구격자_유소년 = 거주인구격자_유소년.fillna(0)
거주인구격자_유소년.head(2)

,gid,val,geometry
0,다사551085,0.0,"MULTIPOLYGON (((126.99422 37.17418, 126.99421 ..."
1,다사551086,0.0,"MULTIPOLYGON (((126.99421 37.17508, 126.99420 ..."


In [72]:
거주인구격자_유소년.loc[0,"geometry"].centroid.coords[0]

(126.99477586013964, 37.174635450576154)

In [73]:
def ppl_living_inside(geo_obj):
    
    global 거주인구격자_유소년
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_유소년.iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 200:
            ret_cnt+=cnt
            
    return ret_cnt

data['격자내 유소년 인구'] = data['geometry'].apply(ppl_living_inside)
data.head(3)

,gid,accident_cnt,격자내 유동인구,격자내 유소년 인구,격자내 학원 수,geometry
0,다사561083,1,43.435676,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,9.633417,0.0,0,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,0.215500,0.0,0,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


In [23]:
data.to_file(os.path.join(path,'GDF_for_linear_regression.geojson'), driver='GeoJSON')

In [24]:
data

,gid,accident_cnt,geometry,격자내 유동인구,격자내 유소년 인구
0,다사561083,1,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ...",43.435676,0.0
1,다사564096,1,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ...",9.633417,0.0
2,다사566102,1,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ...",0.215500,0.0
3,다사583093,1,"MULTIPOLYGON (((127.03022 37.18154, 127.03021 ...",0.662000,0.0
4,다사584072,1,"MULTIPOLYGON (((127.03146 37.16262, 127.03145 ...",8.104333,0.0
...,...,...,...,...,...
340,다사633033,1,"MULTIPOLYGON (((127.08684 37.12767, 127.08683 ...",18.696500,0.0
341,다사633049,1,"MULTIPOLYGON (((127.08676 37.14209, 127.08676 ...",74.573667,47.0
342,다사634036,1,"MULTIPOLYGON (((127.08795 37.13038, 127.08794 ...",63.209333,6.0
343,다사635067,1,"MULTIPOLYGON (((127.08892 37.15832, 127.08892 ...",85.970583,0.0


### feature : 후보격자 중심에서 300m 내에 있는 학원 (1개 칼럼)

In [26]:
학원_및_교습소_현황["업종구분명"].value_counts()

학교교과교습학원    1590
평생직업교육학원     106
교습소           90
Name: 업종구분명, dtype: int64

In [10]:
drop_idx = 학원_및_교습소_현황[학원_및_교습소_현황["교습과정명"].isnull()].index

학원_및_교습소_현황.drop(drop_idx,inplace=True)

In [36]:
for k in 학원_및_교습소_현황[학원_및_교습소_현황["업종구분명"]=="학교교과교습학원"]["교습과정명"].values:
    print(k)

종합(국,수,사,과,논술)
피아노(체르니40이상)
관현악(바이올린,플룻)
피아노(바이엘)
수학(고등)
영어
피아노(체르니100)
종합(초등)
종합(중등)
영어(고등)
피아노(체르니30)
중등수학2
스쿨수학1,2,3,4,5
한국사1,2,3,4,5
피아노
수학+논술저학년
일본어1,2,3,4,5
체르니100
플룻
종합(국,영,수,사,과)
피아노 단계+1
피아노고급
수학초등4
영어기초반
영어고급반
영어회화특강반
고등영어1
영어똑똑1,2,3,4,5
한자1,2,3,4,5
피아노 단계+2
영어중등심화반
영어(중등)
중등수학
중등영어2
수학v1,2,3,4,5
사회1,2,3,4,5
언어중등
고등수학
영어초등1
수학초등1
독서실1개월
와락수학(P)1,2,3,4,5
수학1,2,3,4,5
와락사회1,2,3,4,5
피아노 중급
피아노중급(하)
바이올린
수학초등 심화반
영어(고등)
수학+전과목종합반
영어초등4
독서(1)1,2,3,4,5
스쿨국어(p)1,2,3,4,5
전과목(초등)
피아노(체르니100)
영어초등3
영역국어1,2,3,4,5
똑똑(P)1,2,3,4,5
써밋중등1,2,3
수학
기타
피아노중급
스쿨국어(2)1,2,3,4,5
와락국어1,2,3,4,5
미술
피아노초급(바이엘)
피아노(체르니30)
수학(초중등)
피아노초급
종합초등1
영어초등2
영어(초중등)
스쿨수학(p)1,2,3,4,5
국어1,2,3,4,5
영어1,2,3,4,5
피아노초급
수학단과반
피아노중급
피아노고급2
중등영어1
와락과학1,2,3,4,5
수학+논술
국어(고등)
첼로
영어(초등)
문법1,2,3,4,5
사회(P)1,2,3,4,5
일본어(P)1,2,3,4,5
우쿨렐레
체르니30
영어리딩,문법특강반
중등종합2
성악(입시)
초등수학2
한자(P)1,2,3,4,5
써밋초등1,2
수학+전과목저학년
피아노초급
플룻
영어중등반
수학중등 기초반
중등수학1
사고력1,2,3,4,5
해피톡(p)1,2,3,4,5
와락영어(P)11,2,3,4,5
언어초등
바이올린
영어기초심화반
피아노고급1
기타
초등수학3
독서실1일
과학1,2,3,4,5
듣

In [11]:
drop_list = ["중등","고등","대학","일반","성인","입시"]

def drop_check(name):
    if any([True for element in drop_list if element in name]):
        return True
    return False

drop_idx = 학원_및_교습소_현황['교습과정명'].map(drop_check)

학원_및_교습소_현황 = 학원_및_교습소_현황[drop_idx!=True].copy()

In [12]:
관심_학원_및_교습소_현황 = 학원_및_교습소_현황[학원_및_교습소_현황["업종구분명"]=="학교교과교습학원"].copy()
관심_학원_및_교습소_현황.head(2)

,업종구분명,교습과정명,시설위치_경도,시설위치_위도
3,학교교과교습학원,"종합(국,수,사,과,논술)",127.048919,37.146279
4,학교교과교습학원,피아노(체르니40이상),127.048919,37.146279


In [74]:
def hakwon_inside(geo_obj):
    
    global 관심_학원_및_교습소_현황
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_1,_2,c_y,c_x) in 관심_학원_및_교습소_현황.iloc:
        if haversine((x,y),(c_x,c_y),unit="m") <= 300:
            ret_cnt+=1
    
    return ret_cnt

data['격자내 학원 수'] = data['geometry'].apply(hakwon_inside)
data.head(3)

,gid,accident_cnt,격자내 유동인구,격자내 유소년 인구,격자내 학원 수,geometry
0,다사561083,1,43.435676,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,9.633417,0.0,0,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,0.215500,0.0,0,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


In [81]:
data.to_file(os.path.join(path,'data.geojson'), driver='GeoJSON')

In [16]:
data = gpd.read_file(os.path.join(path,'data.geojson'))
data.head()

,gid,accident_cnt,격자내 유동인구,격자내 유소년 인구,격자내 학원 수,격자내 유소년 인구(100m),격자내 학원 수(100m),geometry
0,다사561083,1,43.435676,0.0,0,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,9.633417,0.0,0,0.0,0,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,0.215500,0.0,0,0.0,0,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."
3,다사583093,1,0.662000,0.0,0,0.0,0,"MULTIPOLYGON (((127.03022 37.18154, 127.03021 ..."
4,다사584072,1,8.104333,0.0,0,0.0,0,"MULTIPOLYGON (((127.03146 37.16262, 127.03145 ..."


# 추가 주문 (100m)

In [17]:
def ppl_living_inside_100(geo_obj):
    
    global 거주인구격자_유소년
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_유소년.iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=cnt
            
    return ret_cnt

data['격자내 유소년 인구(100m)'] = data['geometry'].apply(ppl_living_inside_100)
data.head(3)

,gid,accident_cnt,격자내 유동인구,격자내 유소년 인구,격자내 학원 수,격자내 유소년 인구(100m),격자내 학원 수(100m),geometry
0,다사561083,1,43.435676,0.0,0,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,9.633417,0.0,0,0.0,0,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,0.215500,0.0,0,0.0,0,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


In [18]:
def hakwon_inside_100(geo_obj):
    
    global 관심_학원_및_교습소_현황
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_1,_2,c_y,c_x) in 관심_학원_및_교습소_현황.iloc:
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=1
    
    return ret_cnt

data['격자내 학원 수(100m)'] = data['geometry'].apply(hakwon_inside_100)
data.head(3)

,gid,accident_cnt,격자내 유동인구,격자내 유소년 인구,격자내 학원 수,격자내 유소년 인구(100m),격자내 학원 수(100m),geometry
0,다사561083,1,43.435676,0.0,0,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."
1,다사564096,1,9.633417,0.0,0,0.0,0,"MULTIPOLYGON (((127.00880 37.18416, 127.00879 ..."
2,다사566102,1,0.215500,0.0,0,0.0,0,"MULTIPOLYGON (((127.01101 37.18958, 127.01101 ..."


In [19]:
data.to_file(os.path.join(path,'data.geojson'), driver='GeoJSON')

In [62]:
total["grid_유소년인구_100"] = data["격자내 유소년 인구(100m)"].copy()
total["grid_학원수_100"] = data["격자내 학원 수(100m)"].copy()
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,grid_유동인구,grid_유소년인구,grid_학원수,geometry,grid_유소년인구_100,grid_학원수_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ...",0.0,0


In [82]:
total = total.rename(columns={"grid_유동인구":"유동인구_sum",
                     "grid_유소년인구":"유소년인구_count_200",
                      "grid_학원수":"학원수_count_300",
                     "grid_유소년인구_100":"유소년인구_count_100",
                     "grid_학원수_100":"학원수_count_100"})

total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,유소년인구_count_200,학원수_count_300,geometry,유소년인구_count_100,학원수_count_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ...",0.0,0


In [ ]:
total.to_file(os.path.join(path,'total.geojson'), driver='GeoJSON')

# 칼럼 추가 무조건

In [84]:
total = gpd.read_file(os.path.join(path,'total.geojson'))
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,유소년인구_count_200,학원수_count_300,유소년인구_count_100,학원수_count_100,geometry
0,다사561083,1,0,0,0,0,0,2,2,43.435676,0.0,0,0.0,0,"MULTIPOLYGON (((127.00549 37.17243, 127.00549 ..."


### 1. 동 (원핫)

In [95]:
def dong_assigned(geo_obj):
    
    global 법정경계_읍면동
    
    point = geo_obj.centroid
    
    for line in 법정경계_읍면동.iloc:
        dong = line.geometry
        if dong.contains(point):
            return line.EMD_KOR_NM
    return np.nan

동_구분=pd.get_dummies(total["geometry"].map(dong_assigned),drop_first=True)
total = total.join(동_구분)
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,...,수청동,양산동,오산동,외삼미동,원동,은계동,지곶동,청학동,청호동,탑동
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,0,0


### 2. 사고격자 주변 주정차 단속 건수 (300m)

In [119]:
def joojeong(geo_obj):
    
    global 주정차단속
    
    cnt=0
    for line in 주정차단속.iloc:
        y,x = line[-3],line[-2]

        if haversine((x,y),(geo_obj.centroid.coords[0][::-1]),unit="m")<=300:
            cnt+=1
    return cnt

total["주정차_count_300"] = total["geometry"].map(joojeong)
total.head(1)

,gid,accident_cnt,유치원_count_300,유치원_count_300,초등학교_count_300,방지턱_count_300,불법주정차_cam_count_300,정류장_count_100,정류장_count_300,유동인구_sum,...,오산원당초통학구역,오산원일초통학구역,오산초가수초공동통학구역,오산초통학구역,운산초통학구역,운암초통학구역,원동초통학구역,필봉초통학구역,화성초통학구역,주정차_count_300
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,0,0


### 3. 사고격자 주변 차량 등록 수 (200m)

In [105]:
def car_registered(geo_obj):
    
    global 차량등록현황_격자
    
    y,x = geo_obj.centroid.coords[0]
    cnt=0
    for line in 차량등록현황_격자[차량등록현황_격자.car_cnt>0].iloc:
        _,val,obj_ = line

        if haversine((x,y),(obj_.centroid.coords[0][::-1]),unit="m")<=200:
            cnt+=val
    return cnt

total["차량등록_count_200"] = total["geometry"].map(car_registered)
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,...,양산동,오산동,외삼미동,원동,은계동,지곶동,청학동,청호동,탑동,차량등록_count_200
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,0,29


### 4. 사고격자 주변 신호등 수 (격자 내부)

In [109]:
def traffic_light(geo_obj):
    
    global 신호등

    cnt = 0
    for line in 신호등.iloc:
        _,obj = line
        if geo_obj.contains(obj):
            cnt+=1
    return cnt

total["신호등_count_100"] = total["geometry"].map(traffic_light)
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,...,오산동,외삼미동,원동,은계동,지곶동,청학동,청호동,탑동,차량등록_count_200,신호등_count_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,29,0


### 5. 사고격자 주변 횡단보도 수 (격자내부)

In [111]:
def cross_walk(geo_obj):
    
    global 횡단보도

    cnt = 0
    for line in 횡단보도.iloc:
        _,obj = line
        if geo_obj.contains(obj):
            cnt+=1
    return cnt

total["횡단보도_count_100"] = total["geometry"].map(cross_walk)
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,...,외삼미동,원동,은계동,지곶동,청학동,청호동,탑동,차량등록_count_200,신호등_count_100,횡단보도_count_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,29,0,0


### 6. 사고격자가 속한 초등학교 학군? (원핫)

In [114]:
def tonghakgu(geo_obj):
    
    global 초등학교_통학구
    
    point = geo_obj.centroid
    
    for line in 초등학교_통학구.iloc:
        dong = line.geometry
        if dong.contains(point):
            return line["학구명"]
    return np.nan

학구_구분=pd.get_dummies(total["geometry"].map(tonghakgu),drop_first=True)
total = total.join(학구_구분)
total.head(1)

,gid,accident_cnt,어린이집_count,유치원_count,초등학교_count,방지턱_count,불법주정차_cam_count,정류장_count_100,정류장_count_300,유동인구_sum,...,오산대원초통학구역,오산원당초통학구역,오산원일초통학구역,오산초가수초공동통학구역,오산초통학구역,운산초통학구역,운암초통학구역,원동초통학구역,필봉초통학구역,화성초통학구역
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,0,0


In [115]:
total.to_file(os.path.join(path,'total.geojson'), driver='GeoJSON')

### 7. 사고 격자 내 유소년 말고 다른 인구?

In [124]:
def total_living_inside_100(geo_obj):
    
    global 거주인구격자_총인구
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_총인구[거주인구격자_총인구.val>0].iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=cnt
            
    return ret_cnt

total['총인구_count_100'] = total['geometry'].apply(total_living_inside_100)
total.head(1)

,gid,accident_cnt,유치원_count_300,유치원_count_300,초등학교_count_300,방지턱_count_300,불법주정차_cam_count_300,정류장_count_100,정류장_count_300,유동인구_sum,...,오산원일초통학구역,오산초가수초공동통학구역,오산초통학구역,운산초통학구역,운암초통학구역,원동초통학구역,필봉초통학구역,화성초통학구역,주정차_count_300,총인구_count_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,0,0.0


In [127]:
def doable_living_inside_100(geo_obj):
    
    global 거주인구격자_생산가능
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_생산가능[거주인구격자_생산가능.val>0].iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=cnt
            
    return ret_cnt

def old_living_inside_100(geo_obj):
    
    global 거주인구격자_고령
    
    ret_cnt = 0
    y,x = geo_obj.centroid.coords[0]
    
    for (_,cnt,square) in 거주인구격자_고령[거주인구격자_고령.val>0].iloc:
        c_y,c_x = square.centroid.coords[0]
        
        if haversine((x,y),(c_x,c_y),unit="m") <= 100:
            ret_cnt+=cnt
            
    return ret_cnt


total['생산가능인구_count_100'] = total['geometry'].apply(doable_living_inside_100)
total['고령가능인구_count_100'] = total['geometry'].apply(old_living_inside_100)
total.head(1)

,gid,accident_cnt,유치원_count_300,유치원_count_300,초등학교_count_300,방지턱_count_300,불법주정차_cam_count_300,정류장_count_100,정류장_count_300,유동인구_sum,...,오산초통학구역,운산초통학구역,운암초통학구역,원동초통학구역,필봉초통학구역,화성초통학구역,주정차_count_300,총인구_count_100,생산가능인구_count_100,고령가능인구_count_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [120]:
total.columns

Index(['gid', 'accident_cnt', '유치원_count_300', '유치원_count_300',
       '초등학교_count_300', '방지턱_count_300', '불법주정차_cam_count_300',
       '정류장_count_100', '정류장_count_300', '유동인구_sum', '유소년인구_count_200',
       '학원수_count_300', '유소년인구_count_100', '학원수_count_100', 'geometry', '가장동',
       '갈곶동', '고현동', '궐동', '금암동', '내삼미동', '누읍동', '벌음동', '부산동', '서동', '서랑동',
       '세교동', '수청동', '양산동', '오산동', '외삼미동', '원동', '은계동', '지곶동', '청학동', '청호동',
       '탑동', '차량등록_count_200', '신호등_sum', '횡단보도_sum', '광성초양산초병점초공동통학구역',
       '광성초통학구역', '금암초통학구역', '대호초성산초공동통학구역', '대호초통학구역', '매홀초삼미분교수청초필봉초공동통학구역',
       '매홀초삼미분교장통학구역', '매홀초통학구역', '문시초통학구역', '성산초통학구역', '성호초운암초공동통학구역',
       '성호초통학구역', '세미초통학구역', '수청초통학구역', '양산초통학구역', '오산고현초오산원당초공동통학구역',
       '오산고현초통학구역', '오산대원초원동초공동통학구역', '오산대원초통학구역', '오산원당초통학구역', '오산원일초통학구역',
       '오산초가수초공동통학구역', '오산초통학구역', '운산초통학구역', '운암초통학구역', '원동초통학구역', '필봉초통학구역',
       '화성초통학구역', '주정차_count_300'],
      dtype='object')

In [118]:
total = total.rename(columns={"어린이집_count":"유치원_count_300",
                             "유치원_count":"유치원_count_300",
                             "초등학교_count":"초등학교_count_300",
                             "방지턱_count":"방지턱_count_300",
                             "불법주정차_cam_count":"불법주정차_cam_count_300",
                             "신호등_count_100":"신호등_sum",
                             "횡단보도_count_100":"횡단보도_sum"})
total.head(1)

,gid,accident_cnt,유치원_count_300,유치원_count_300,초등학교_count_300,방지턱_count_300,불법주정차_cam_count_300,정류장_count_100,정류장_count_300,유동인구_sum,...,오산대원초통학구역,오산원당초통학구역,오산원일초통학구역,오산초가수초공동통학구역,오산초통학구역,운산초통학구역,운암초통학구역,원동초통학구역,필봉초통학구역,화성초통학구역
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0,0,0


In [128]:
total = total.rename(columns={"고령가능인구_count_100":"고령인구_count_100"})
total.head(1)

,gid,accident_cnt,유치원_count_300,유치원_count_300,초등학교_count_300,방지턱_count_300,불법주정차_cam_count_300,정류장_count_100,정류장_count_300,유동인구_sum,...,오산초통학구역,운산초통학구역,운암초통학구역,원동초통학구역,필봉초통학구역,화성초통학구역,주정차_count_300,총인구_count_100,생산가능인구_count_100,고령인구_count_100
0,다사561083,1,0,0,0,0,0,2,2,43.435676,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [130]:
total.to_file(os.path.join(path,'total.geojson'), driver='GeoJSON')

In [132]:
total_df = pd.DataFrame(total)
total_df.to_csv(os.path.join(path,'total.csv'),index=False)